In [3]:
!pip install boto3

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
!head ./amazon20000.csv

marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
US,12190288,R3FU16928EP5TC,B00AYB1482,668895143,Enlightened: Season 1,Digital_Video_Download,5,0,0,N,Y,I loved it and I wish there was a season 3,I loved it and I wish there was a season 3... I watched season 2 and loved that as well!,2015-08-31
US,30549954,R1IZHHS1MH3AQ4,B00KQD28OM,246219280,Vicious,Digital_Video_Download,5,0,0,N,Y,As always it seems that the best shows come from England,"As always it seems that the best shows come from England. best of the best without words, i cant wait to watch season two.",2015-08-31
US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of things right. Yes, the librarian character played by Marcia Gay Harden is stereotypical and played a bit heavy-handed. But 

In [3]:
df = pd.read_csv('amazon20000.csv')
df.shape

(19742, 15)

In [4]:
df_dropna = df.dropna()

In [5]:
df_dropna.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12190288,R3FU16928EP5TC,B00AYB1482,668895143,Enlightened: Season 1,Digital_Video_Download,5,0,0,N,Y,I loved it and I wish there was a season 3,I loved it and I wish there was a season 3... ...,2015-08-31
1,US,30549954,R1IZHHS1MH3AQ4,B00KQD28OM,246219280,Vicious,Digital_Video_Download,5,0,0,N,Y,As always it seems that the best shows come fr...,As always it seems that the best shows come fr...,2015-08-31
2,US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of...",2015-08-31
3,US,27072354,R7HOOYTVIB0DS,B008LOVIIK,239012694,Masterpiece: Inspector Lewis Season 5,Digital_Video_Download,5,0,0,N,Y,Five Stars,excellant this is what tv should be,2015-08-31
4,US,26939022,R1XQ2N5CDOZGNX,B0094LZMT0,535858974,On The Waterfront,Digital_Video_Download,5,0,0,N,Y,Brilliant film from beginning to end,Brilliant film from beginning to end. All of t...,2015-08-31


In [6]:
df_cleaned_and_filtered = df_dropna.query('helpful_votes > 0')
df_cleaned_and_filtered.shape

(2019, 15)

In [7]:
df_cleaned_and_filtered['pct_helpful_votes'] = df_cleaned_and_filtered['helpful_votes'] / df_cleaned_and_filtered['total_votes']
df_cleaned_and_filtered.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(2019, 16)

In [8]:
df_cleaned_and_filtered.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,pct_helpful_votes
2,US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of...",2015-08-31,0.944444
5,US,4772040,R1HCST57W334KN,B0112OSOQE,38517795,Rick and Morty Season 2,Digital_Video_Download,5,5,6,N,Y,Best show on TV right now,If you don't like this show. Go back to your n...,2015-08-31,0.833333
6,US,12910040,R32BUTYQS1ZJBQ,B000NPE5SA,373323715,Africa Screams,Digital_Video_Download,4,1,1,N,Y,Very funny. A typical mid 50's comedy,Very funny. A typical mid 50's comedy.,2015-08-31,1.000000
18,US,47856888,R1ZHHD1GTHGMNK,B005U9RM0W,728197314,Grimm Season 1,Digital_Video_Download,4,1,1,N,Y,If you're a sucker for supernatural series the...,If you're a sucker for supernatural series the...,2015-08-31,1.000000
55,US,52684121,R203639GCLF7XV,B00MYLVPFM,773772186,Amazing Race 8,Digital_Video_Download,1,5,5,N,Y,Like many other viewers I felt the Family edit...,Like many other viewers I felt the Family edit...,2015-08-31,1.000000


In [9]:
df_cleaned_and_filtered['is_helpful'] = df_cleaned_and_filtered['pct_helpful_votes'] > 0.75
df_cleaned_and_filtered.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(2019, 17)

In [10]:
df_cleaned_and_filtered.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,pct_helpful_votes,is_helpful
2,US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of...",2015-08-31,0.944444,True
5,US,4772040,R1HCST57W334KN,B0112OSOQE,38517795,Rick and Morty Season 2,Digital_Video_Download,5,5,6,N,Y,Best show on TV right now,If you don't like this show. Go back to your n...,2015-08-31,0.833333,True
6,US,12910040,R32BUTYQS1ZJBQ,B000NPE5SA,373323715,Africa Screams,Digital_Video_Download,4,1,1,N,Y,Very funny. A typical mid 50's comedy,Very funny. A typical mid 50's comedy.,2015-08-31,1.000000,True
18,US,47856888,R1ZHHD1GTHGMNK,B005U9RM0W,728197314,Grimm Season 1,Digital_Video_Download,4,1,1,N,Y,If you're a sucker for supernatural series the...,If you're a sucker for supernatural series the...,2015-08-31,1.000000,True
55,US,52684121,R203639GCLF7XV,B00MYLVPFM,773772186,Amazing Race 8,Digital_Video_Download,1,5,5,N,Y,Like many other viewers I felt the Family edit...,Like many other viewers I felt the Family edit...,2015-08-31,1.000000,True


## TODO:  Train on just `review_body` and `star_rating`

In [11]:
df_cleaned_and_filtered = df_cleaned_and_filtered[['review_body', 'star_rating', 'is_helpful']]
df_cleaned_and_filtered.shape

(2019, 3)

In [12]:
df_cleaned_and_filtered.to_csv(path_or_buf='amazon-digital-video-download-cleaned-and-filtered.csv', index=False)

#### TODO:  Hold out 5-10% to test separate from AutoML 

In [13]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)

In [14]:
#X_train.shape

In [15]:
#X_train.to_csv(path_or_buf='amazon-digital-video-download-5000-cleaned-and-filtered.csv', index=False)

In [16]:
prefix = 'xgboost-reviews/autopilot'

sess.upload_data(path='amazon-digital-video-download-cleaned-and-filtered.csv', bucket=bucket, key_prefix=prefix + '/input')

's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/input/amazon-digital-video-download-cleaned-and-filtered.csv'

In [17]:
!aws s3 ls $bucket/$prefix/input/

2020-02-19 03:11:03     508200 amazon-digital-video-download-cleaned-and-filtered.csv


In [18]:
max_candidates = 25

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/input'.format(bucket, prefix)
        }
      },
      'TargetAttributeName': 'is_helpful'
    }
]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket, prefix)
}

#problem_type = 'Regression'

#auto_ml_job_objective = {
#    'MetricName': 'MSE'
#}

## Launching the SageMaker AutoPilot job

We can now launch the job by calling the `create_auto_ml_job` API.

In [19]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

AutoMLJobName: automl-dm-19-03-11-03


In [20]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType=problem_type,
#                      AutoMLJobObjective=auto_ml_job_objective,
                      RoleArn=role)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-19-03-11-03',
 'ResponseMetadata': {'RequestId': 'b79ff9b1-2789-4291-a49c-e75a07ff10f5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b79ff9b1-2789-4291-a49c-e75a07ff10f5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Wed, 19 Feb 2020 03:11:03 GMT'},
  'RetryAttempts': 0}}

### Tracking the progress of the AutoPilot job
SageMaker AutoPilot job consists of four high-level steps : 
* Data Preprocessing, where the dataset is split into train and validation sets.
* Recommending Pipelines, where the dataset is analyzed and SageMaker AutoPilot comes up with a list of ML pipelines that should be tried out on the dataset.
* Automatic Feature Engineering, where SageMaker AutoPilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* ML pipeline selection and hyperparameter tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [21]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
Data analysis complete
{'AutoMLJobName': 'automl-dm-19-03-11-03', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-19-03-11-03', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/input'}}, 'TargetAttributeName': 'is_helpful'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output'}, 'RoleArn': 'arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T13

In [22]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

InProgress
FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress

In [23]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

InProgress
ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
Completed MaxCandidatesReached
Model tuning complete
{'AutoMLJobName': 'automl-dm-19-03-11-03', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-19-03-11-03', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/input'}}, 'TargetAttributeName': 'is_helpful'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output'}, 'RoleArn': 'arn:aws:iam::835319576252

### Viewing all candidates explored by SageMaker AutoPilot
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [24]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

0  tuning-job-1-9e3ed4d4341f4794bf-020-bd36a133  0.7741940021514893
1  tuning-job-1-9e3ed4d4341f4794bf-002-ca1eba2d  0.7642679810523987
2  tuning-job-1-9e3ed4d4341f4794bf-001-5fc2e7eb  0.7642679810523987
3  tuning-job-1-9e3ed4d4341f4794bf-006-7102eff7  0.7642679810523987
4  tuning-job-1-9e3ed4d4341f4794bf-005-0532b30f  0.7642679810523987
5  tuning-job-1-9e3ed4d4341f4794bf-024-4cf3e8d4  0.7642679810523987
6  tuning-job-1-9e3ed4d4341f4794bf-003-dcd69de5  0.7642679810523987
7  tuning-job-1-9e3ed4d4341f4794bf-004-b0532f2d  0.7642679810523987
8  tuning-job-1-9e3ed4d4341f4794bf-007-e494c4bd  0.7493799924850464
9  tuning-job-1-9e3ed4d4341f4794bf-010-4b01691c  0.7444170117378235


## Inspect SageMaker AutoPilot trials with Amazon SageMaker Experiments
SageMaker AutoPilot automatically creates a new experiment, and pushes information for each trial. 

In [25]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
df

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,_tuning_objective_metric,binary_classifier_model_selection_criteria,l1,...,train:accuracy - Count,SageMaker.ModelName,SageMaker.ModelPrimary.DataUrl,SageMaker.ModelPrimary.Image,processor_module,sagemaker_program,sagemaker_submit_directory,input_channel_mode,job_name,label_col
0,tuning-job-1-9e3ed4d4341f4794bf-024-4cf3e8d4-a...,tuning-job-1-9e3ed4d4341f4794bf-024-4cf3e8d4-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,382416733822.dkr.ecr.us-east-1.amazonaws.com/l...,1.0,ml.m5.4xlarge,50.0,validation:binary_classification_accuracy,loss_function,6.744127e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tuning-job-1-9e3ed4d4341f4794bf-025-1a6e8908-a...,tuning-job-1-9e3ed4d4341f4794bf-025-1a6e8908-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tuning-job-1-9e3ed4d4341f4794bf-022-41a0c65b-a...,tuning-job-1-9e3ed4d4341f4794bf-022-41a0c65b-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tuning-job-1-9e3ed4d4341f4794bf-023-40babbf1-a...,tuning-job-1-9e3ed4d4341f4794bf-023-40babbf1-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tuning-job-1-9e3ed4d4341f4794bf-021-882ee843-a...,tuning-job-1-9e3ed4d4341f4794bf-021-882ee843-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tuning-job-1-9e3ed4d4341f4794bf-020-bd36a133-a...,tuning-job-1-9e3ed4d4341f4794bf-020-bd36a133-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,tuning-job-1-9e3ed4d4341f4794bf-017-29e56116-a...,tuning-job-1-9e3ed4d4341f4794bf-017-29e56116-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,tuning-job-1-9e3ed4d4341f4794bf-019-eae39640-a...,tuning-job-1-9e3ed4d4341f4794bf-019-eae39640-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,tuning-job-1-9e3ed4d4341f4794bf-016-ed8829e9-a...,tuning-job-1-9e3ed4d4341f4794bf-016-ed8829e9-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tuning-job-1-9e3ed4d4341f4794bf-018-3619fa9d-a...,tuning-job-1-9e3ed4d4341f4794bf-018-3619fa9d-a...,arn:aws:sagemaker:us-east-1:835319576252:train...,382416733822.dkr.ecr.us-east-1.amazonaws.com/l...,1.0,ml.m5.4xlarge,50.0,validation:binary_classification_accuracy,loss_function,1.518917e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Viewing notebooks generated by SageMaker AutoPilot
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data exploration,
* Candidate definition.

In [26]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

{'AutoMLJobName': 'automl-dm-19-03-11-03', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-19-03-11-03', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/input'}}, 'TargetAttributeName': 'is_helpful'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output'}, 'RoleArn': 'arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 25, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 2, 19, 3, 11, 3, 754000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 2, 19, 3, 45, 29, 46000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 19, 3, 45, 29, 77000, tzinfo=tzlocal()), 'BestCandidate': {'CandidateName': 'tuning-job-1-

### Let's copy all of the generated resources including the two notebooks.

In [27]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

's3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/sagemaker-automl-candidates/pr-1-ad4d16b8ef504ef0815d290d266acc52b94acd103c7c41c6a9fe99155d'

In [28]:
!rm -rf ./generated_module
!rm -rf ./notebooks
!aws s3 cp --recursive $generated_resources .

download: s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/sagemaker-automl-candidates/pr-1-ad4d16b8ef504ef0815d290d266acc52b94acd103c7c41c6a9fe99155d/generated_module/MANIFEST.in to generated_module/MANIFEST.in
download: s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/sagemaker-automl-candidates/pr-1-ad4d16b8ef504ef0815d290d266acc52b94acd103c7c41c6a9fe99155d/generated_module/candidate_data_processors/dpp4.py to generated_module/candidate_data_processors/dpp4.py
download: s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/sagemaker-automl-candidates/pr-1-ad4d16b8ef504ef0815d290d266acc52b94acd103c7c41c6a9fe99155d/generated_module/candidate_data_processors/dpp3.py to generated_module/candidate_data_processors/dpp3.py
download: s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/sagemaker-automl-candidates/pr-1-ad4d16b8

### _In the file view, open the `notebooks` and `generated_module` folders.  Lots of useful information in there!_

## Deploying the best candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [30]:
best_candidate

{'CandidateName': 'tuning-job-1-9e3ed4d4341f4794bf-020-bd36a133',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.7741940021514893},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:835319576252:processing-job/db-1-bd0f1a98656a47f9801e6f1c1412a5ecd9912d38493640679da272654c',
   'CandidateStepName': 'db-1-bd0f1a98656a47f9801e6f1c1412a5ecd9912d38493640679da272654c'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:835319576252:training-job/automl-dm--dpp7-1-1c9706196d7b4e15b98058c1d6d61d47180a265b60a84',
   'CandidateStepName': 'automl-dm--dpp7-1-1c9706196d7b4e15b98058c1d6d61d47180a265b60a84'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:835319576252:transform-job/automl-dm--dpp7-rpb-1-335f2ea98dc84f3dad28aa136c1d73d35b54

We can see the containers and models composing the Inference Pipeline.

In [31]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/data-processor-models/automl-dm--dpp7-1-1c9706196d7b4e15b98058c1d6d61d47180a265b60a84/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3
s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/tuning/automl-dm--dpp7-xgb/tuning-job-1-9e3ed4d4341f4794bf-020-bd36a133/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-east-1-835319576252/xgboost-reviews/autopilot/output/automl-dm-19-03-11-03/data-processor-models/automl-dm--dpp7-1-1c9706196d7b4e15b98058c1d6d61d47180a265b60a84/output/model.tar.gz


In [32]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Best candidate model ARN:  arn:aws:sagemaker:us-east-1:835319576252:model/automl-dm-model-19-03-11-03


Let's deploy the pipeline.

In [33]:
# EndpointConfig name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
ep_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(ep_name)
print(variant_name)

automl-dm-ep-19-03-46-17
automl-dm-variant-19-03-46-17


In [34]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m4.xlarge',
                                                           'InitialInstanceCount':1,
                                                           'ModelName':model_name,
                                                           'VariantName':variant_name}])

create_endpoint_response = sm.create_endpoint(EndpointName=ep_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

arn:aws:sagemaker:us-east-1:835319576252:endpoint/automl-dm-ep-19-03-46-17


In [ ]:
%%time
sm.get_waiter('endpoint_in_service').wait(EndpointName=ep_name)

In [ ]:
resp = sm.describe_endpoint(EndpointName=ep_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

## Scoring the Best Candidate
Let's predict and score the validation set. We'll compute metrics ourselves just for fun.

In [5]:
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

In [13]:
#ep_name = 'automl-dm-ep-19-03-46-17'

# TODO:  Figure out the param order

#csv_line_predict_0 = """US,12190288,R3FU16928EP5TC,B00AYB1482,668895143,Enlightened: Season 1,Digital_Video_Download,5,0,N,Y,I loved it and I wish there was a season 3,I loved it and I wish there was a season 3... I watched season 2 and loved that as well!,8/31/15"""
csv_line_predict_0 = """I loved it and I wish there was a season 3... I watched season 2 and loved that as well!,5"""
response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_0)

response_body = response['Body'].read().decode("utf-8").strip()
response_body

'True'

In [12]:
#csv_line_predict_17 = """US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,18,N,Y,Charming movie,This movie isn't perfect  but it gets a lot of things right. Yes  the librarian character played by Marcia Gay Harden is stereotypical and played a bit heavy-handed. But the universal nature of the story  the beautiful setting  and the likability of the characters overcome this flaw. The quote at the end brought tears to my eyes.  If you want to take a break from Hollywood's standard fare of dark  violent  or stupid movies  then give this a try. It is is entertaining and thoughtful.,8/31/15"""
csv_line_predict_17 = """This movie isn't perfect  but it gets a lot of things right. Yes  the librarian character played by Marcia Gay Harden is stereotypical and played a bit heavy-handed. But the universal nature of the story  the beautiful setting  and the likability of the characters overcome this flaw. The quote at the end brought tears to my eyes.  If you want to take a break from Hollywood's standard fare of dark  violent  or stupid movies  then give this a try. It is is entertaining and thoughtful.,"""
response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_17)

response_body = response['Body'].read().decode("utf-8").strip()
response_body

'True'

In [ ]:
# with open('holdout-test-dataset.csv') as f:
#     lines = f.readlines()
#     for l in lines[1:]:   # Skip header
#         l = l.split(',')  # Split CSV line into features
#         label = l[-1]     # Store 'yes'/'no' label
#         l = l[:-1]        # Remove label
#         l = ','.join(l)   # Rebuild CSV line without label
                
#         response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=l)

#         response = response['Body'].read().decode("utf-8")
 
#         if 'yes' in label:
#             # Sample is positive
#             if 'yes' in response:
#                 # True positive
#                 tp=tp+1
#             else:
#                 # False negative
#                 fn=fn+1
#         else:
#             # Sample is negative
#             if 'no' in response:
#                 # True negative
#                 tn=tn+1
#             else:
#                 # False positive
#                 fp=fp+1
#         count = count+1
#         if (count % 100 == 0):   
#             sys.stdout.write(str(count)+' ')

In [ ]:
# #Confusion matrix
# print ("%d %d" % (tn, fp))
# print ("%d %d" % (fn, tp))

# accuracy  = (tp+tn)/(tp+tn+fp+fn)
# precision = tp/(tp+fp)
# recall    = tn/(tn+fn)
# f1        = (2*precision*recall)/(precision+recall)

# print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))